#### Step 1: Importing Libraries

In [ ]:
import json
import pandas as pd
import glob

#### Step 1: Data Loading

In [ ]:

# Step 1: Get all JSON file names from the folder
files = glob.glob("*.json") # change this to your folder path

print("Found files:", files)

# Step 2: Read and combine them all
dataframes = []

for file in files:
    with open(file, "r") as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        dataframes.append(df)

# Step 3: Combine all into one DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

print("Combined shape:", combined_df.shape)
print(combined_df.head())


Found files: ['energy.json', 'social_and_economic.json', 'electricity.json']


Combined shape: (50922, 11)
   id     name     score                unit   region_name indicator_topic  \
0  DZ  Algeria  1.122612  Millions of people  North Africa          Access   
1  DZ  Algeria  0.947765  Millions of people  North Africa          Access   
2  DZ  Algeria  0.831871  Millions of people  North Africa          Access   
3  DZ  Algeria  0.712877  Millions of people  North Africa          Access   
4  DZ  Algeria  0.623791  Millions of people  North Africa          Access   

    indicator_source                                     indicator_name  \
0  Tracking SDG7/WBG  Energy: Population without access to clean coo...   
1  Tracking SDG7/WBG  Energy: Population without access to clean coo...   
2  Tracking SDG7/WBG  Energy: Population without access to clean coo...   
3  Tracking SDG7/WBG  Energy: Population without access to clean coo...   
4  Tracking SDG7/WBG  Energy: Population without access to clean coo...   

  indicator_group  year                   url  
0   

#### Step 2: Data Cleaning & Formatting

In [ ]:
# Step 1: Rename columns to match internship schema
df = combined_df.rename(columns={
    "id": "country_serial",
    "name": "country",
    "indicator_name": "metric",
    "unit": "unit",
    "indicator_group": "sector",
    "indicator_topic": "sub_sector",
    "indicator_source": "source",
    "url": "source_link",
    "score": "value"
})

# Step 2: Clean "value" column — convert text to numeric
df["value"] = pd.to_numeric(df["value"], errors="coerce")
# This changes "NULL", "N/A", etc. → NaN (missing value)


# Step 3: Pivot data (years → columns)
pivot_df = df.pivot_table(
    index=[
        "country", "country_serial", "metric", "unit",
        "sector", "sub_sector", "source_link", "source"
    ],
    columns="year",
    values="value",
    aggfunc="mean"  # can also use "first" if data is unique
).reset_index()

# Step 4: Flatten the column headers
pivot_df.columns.name = None

# Step 5: Save to CSV
pivot_df.to_csv("africa_energy_data.csv", index=False)


pivot_df.head()



,country,country_serial,metric,unit,sector,sub_sector,source_link,source,2000,2001,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Algeria,DZ,Electricity export (GWh),GWh,Electricity,Supply,/aep/country/algeria,AFREC Database,319.000000,196.000000,...,384.000000,877.000000,641.000000,507.000000,918.764526,833.876404,975.834595,1128.864014,1529.344971,NaN
1,Algeria,DZ,Electricity final consumption (GWh),GWh,Electricity,Supply,/aep/country/algeria,AFREC Database,18592.000000,19664.000000,...,40188.000000,45751.000000,47956.968750,52288.000000,56376.101562,58152.601562,59053.726562,60044.328125,62502.121094,NaN
2,Algeria,DZ,Electricity final consumption per capita (KWh),KWh per capita,Electricity,Supply,/aep/country/algeria,AFREC Database,596.209656,622.433105,...,1048.239624,1169.704102,1202.787354,1287.689941,1364.439453,1384.320435,1383.671143,1385.640747,1421.494507,NaN
3,Algeria,DZ,Electricity generated from biofuels and waste ...,GWh,Electricity,Supply,/aep/country/algeria,AFREC Database,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
4,Algeria,DZ,Electricity generated from fossil fuels (GWh),GWh,Electricity,Supply,/aep/country/algeria,AFREC Database,25358.000000,26556.000000,...,59560.000000,63988.000000,68576.000000,70663.000000,75381.968750,75879.992188,79441.468750,83299.226562,84709.507812,NaN


In [7]:
pivot_df.shape

(1943, 31)

#### Step 3: Data Storage

In [8]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 5.9 MB/s eta 0:00:000m eta 0:00:01


In [10]:
from pymongo import MongoClient

try:
    client = MongoClient("mongodb://localhost:27017/", serverSelectionTimeoutMS=5000)
    client.server_info()  # forces connection test
    print("MongoDB is running and connected!")
except Exception as e:
    print("Connection failed:", e)


MongoDB is running and connected!


In [12]:
from pymongo import MongoClient

# Step 1: Ensure year columns are strings
pivot_df.columns = pivot_df.columns.map(str)

# Step 2: Replace NaN with None (MongoDB does not accept NaN)
pivot_df = pivot_df.where(pd.notnull(pivot_df), None)

# Step 3: Convert DataFrame to list of dicts
records = pivot_df.to_dict(orient="records")

# Step 4: Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["africa_energy_db"]
collection = db["energy_data"]

# Step 5: Insert data
collection.insert_many(records)

# Step 6: Verify
print("Inserted:", collection.count_documents({}), "documents")
print(collection.find_one())



Inserted: 1943 documents
{'_id': ObjectId('68ef1e0f36d3b506fcf02f9a'), 'country': 'Algeria', 'country_serial': 'DZ', 'metric': 'Electricity export (GWh)', 'unit': 'GWh', 'sector': 'Electricity', 'sub_sector': 'Supply', 'source_link': '/aep/country/algeria', 'source': 'AFREC Database', '2000': 319.0, '2001': 196.0, '2002': 259.0, '2003': 212.0, '2004': 197.0, '2005': 275.0, '2006': 298.0, '2007': 273.0, '2008': 323.0, '2009': 362.0, '2010': 803.0, '2011': 799.0, '2012': 985.0, '2013': 384.0, '2014': 877.0, '2015': 641.0, '2016': 507.0, '2017': 918.764526367188, '2018': 833.876403808594, '2019': 975.834594726563, '2020': 1128.86401367188, '2021': 1529.34497070313, '2022': nan}


Step 4: Data Validation

In [15]:
import pandas as pd

# === STEP 1: Load your CSV ===
df = pd.read_csv("africa_energy_data.csv")
print("CSV loaded successfully!")
print("Shape:", df.shape)
print(df.head(), "\n")
print("*********************************************************************************************")

# === STEP 2: Check for missing values per column ===
missing_summary = df.isnull().sum()
print("Missing values per column:")
print(missing_summary[missing_summary > 0], "\n")
print("*********************************************************************************************")

# === STEP 3: Automatically detect year columns ===
year_cols = [col for col in df.columns if col.isdigit()]
print("Detected year columns:", year_cols, "\n")
print("*********************************************************************************************")

# Check rows with missing year values
missing_years = df[df[year_cols].isnull().any(axis=1)]
print(f"Found {len(missing_years)} rows with missing year values.\n")
print("*********************************************************************************************")

# Optional: Save missing records for reporting
missing_years.to_csv("missing_years_report.csv", index=False)

# === STEP 4: Check for missing or blank metric names ===
missing_metrics = df[df["metric"].isnull() | (df["metric"] == "")]
print(f"Rows missing metric names: {len(missing_metrics)}\n")
print("*********************************************************************************************")

# === STEP 5: Check for unit consistency ===
print("Unique units:")
print(df["unit"].unique(), "\n")
print("*********************************************************************************************")

unit_check = df.groupby("metric")["unit"].nunique().reset_index()
inconsistent_units = unit_check[unit_check["unit"] > 1]
print(f"Metrics with inconsistent units: {len(inconsistent_units)}")
if len(inconsistent_units) > 0:
    print(inconsistent_units, "\n")
print("*********************************************************************************************")

# === STEP 6: Check country naming consistency ===
countries = sorted(df["country"].unique())
print(f"Total unique countries: {len(countries)}")
print("Sample countries:", countries[:10], "\n")
print("*********************************************************************************************")

# === STEP 7: Final validation summary ===
summary = {
    "Total rows": len(df),
    "Rows with missing year data": len(missing_years),
    "Rows missing metric names": len(missing_metrics),
    "Metrics with inconsistent units": len(inconsistent_units),
    "Unique countries": len(countries)
}

print("Data Validation Summary:")
for k, v in summary.items():
    print(f"{k}: {v}")



CSV loaded successfully!
Shape: (1943, 31)
   country country_serial                                             metric  \
0  Algeria             DZ                           Electricity export (GWh)   
1  Algeria             DZ                Electricity final consumption (GWh)   
2  Algeria             DZ     Electricity final consumption per capita (KWh)   
3  Algeria             DZ  Electricity generated from biofuels and waste ...   
4  Algeria             DZ      Electricity generated from fossil fuels (GWh)   

             unit       sector sub_sector           source_link  \
0             GWh  Electricity     Supply  /aep/country/algeria   
1             GWh  Electricity     Supply  /aep/country/algeria   
2  KWh per capita  Electricity     Supply  /aep/country/algeria   
3             GWh  Electricity     Supply  /aep/country/algeria   
4             GWh  Electricity     Supply  /aep/country/algeria   

           source          2000          2001  ...          2013  \
0  AF